In [9]:
import polars as pl

data = pl.read_csv("output/output.csv")

In [12]:
from collections import OrderedDict

# Create an ordered dictionary
od = OrderedDict()
od['a'] = 1
od['b'] = 2
od['c'] = 3


for item in od.values():

    print(item)

34675
